In [1]:
import json
from datetime import datetime
from urllib import parse
import requests
import time

from wenshu_utils.cipher import CipherText
from wenshu_utils.des3 import des3decrypt
from wenshu_utils.pageid import PageID
from wenshu_utils.token import RequestVerificationToken

API = "http://120.78.76.198:8000/wenshu"

### Design: query keywords to get document id, then download json files using document id. 

index
s8, value=02: 刑事案件
s8, value=03: 民事案件
s8, value=04: 行政案件
s8, value=05: 赔偿案件
s8, value=10: 执行案件

In [4]:
# keywords for query
query_text_lst = ["人身权利",
"拘役",
"犯罪未遂",
"违法所得",
"扣押",
"偶犯",
"聚众",
"驳回",
"交通事故",
"合法财产",
"鉴定",
"没收",
"注册商标",
"增值税",
"所有权",
"管制",
"投资",
"假冒",
"立功",
"寻衅滋事",
"集资",
"非法经营",
"假药",
"着手",
"附带民事诉讼"]
total = []

In [ ]:
class Wenshu:
#     url : parse.ParseResult = parse.urlparse("http://wenshu.court.gov.cn/website/parse/rest.q4w")
    url = parse.urlparse("http://wenshu.court.gov.cn/website/parse/rest.q4w")
#     url = parse.ParseResult
    
    
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36",
        })

    def _request(self, data: dict) -> requests.Response:
        response = self.session.post(self.url.geturl(), data=data)
        if response.status_code != 200:
            raise Exception(response.status_code)

        json_data = response.json()

        plain_text = des3decrypt(cipher_text=json_data["result"],
                                 key=json_data["secretKey"],
                                 iv=datetime.now().strftime("%Y%m%d"))

        result = json.loads(plain_text)
        return result

    def list_page(self, word):
        """input query, return page list"""
        
        print("Page id: \n", PageID())
        print()

        # the maximum size for one page is 1000
        data = {
            "pageId": PageID(),
            "sortFields": "s50:desc",
            "ciphertext": CipherText(),
            "pageNum": 1,
            "pageSize": 1000,
            "queryCondition": json.dumps([{"key": "s42", "value": "2018"},
                                          {"key": "s33", "value": "上海市"},
                                          {"key": "s8", "value": "02"},
                                          {"key": "s45", "value": word}]),  
            "cfg": "com.lawyee.judge.dc.parse.dto.SearchDataDsoDTO@queryDoc",
            "__RequestVerificationToken": RequestVerificationToken(24),
        }

        result = self._request(data)
#         print("list_page: ", result)
        id_lst = list(result['relWemshu'].keys())
#         print(id_lst)
        print("Query word: ", word)
        print("length: ", len(id_lst))
        print()
        return id_lst

    def detail_page(self, doc_id):
        """input """
        print(doc_id)
        data = {
            # ORIGINAL: 4e00b8ae589b4288a725aabe00c0e683
#             "docId": "4e00b8ae589b4288a725aabe00c0e683",
            "docId": doc_id,
            "ciphertext": CipherText(),
            "cfg": "com.lawyee.judge.dc.parse.dto.SearchDataDsoDTO@docInfoSearch",
            "__RequestVerificationToken": RequestVerificationToken(24),
        }

        result = self._request(data)
#         print("detail_page: ", result)
        return result


#### Collect doc ids returned from queries

In [6]:
demo = Wenshu()
cur_w = 19
for w in range(cur_w, len(query_text_lst)):
    print("working on: {0} index {1}".format(query_text_lst[w], w))
    id_lst_comp = demo.list_page(query_text_lst[w])
    for ids in id_lst_comp:
        total.append(ids)
    time.sleep(60)
#     print(id_lst[-1])
#     demo.detail_page()

working on: 寻衅滋事 index 19
Page id: 
 751398a8fd258224ff4f11a5f0785549

Query word:  寻衅滋事
length:  138



KeyboardInterrupt: 

#### save ids to local 

In [10]:
import pickle
file = open("5151 doc id.p", "wb")
pickle.dump(temp, file)
file.close()

#### Query using ids, write results to /01-web-scraping-raw-data

In [25]:
parsing_doc = Wenshu()
data_path = "./data/01-web-scraping-raw-data"
for i in range(20):
    with open(data_path + list(total_set)[i]+'.json', 'w') as fp:
        sample = parsing_doc.detail_page(list(total_set)[i])
        json.dump(sample, fp, ensure_ascii=False)

a03799afe8c549029028a9620102ee34
detail_page:  {'s1': '罕文军寻衅滋事一审刑事判决书', 's2': '上海市闵行区人民法院', 's3': '915', 's5': 'a03799afe8c549029028a9620102ee34', 's6': '01', 's7': '（2018）沪0112刑初1149号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-08-14', 's41': '2018-09-21', 's22': '上海市闵行区人民法院\n刑事判决书\n（2018）沪0112刑初1149号', 's23': '上海市闵行区人民检察院以沪闵检诉刑诉［2018］993号起诉书指控被告人何文军犯寻衅滋事罪，于2018年8月2日向本院提起公诉。本院依法适用简易程序，实行独任审判，适用认罪认罚从宽制度，公开开庭审理了本案。上海市闵行区人民检察院指派检察员高某出庭支持公诉，被告人何文军及上海市闵行区法律援助中心指派的辩护人应鸿敏到庭参加诉讼。本案现已审理终结', 's25': '公诉机关指控：2018年4月27日下午，被告人何文军等人在上海市闵行区莘朱路ＸＸＸ号“志帆网吧”内，因故与网吧老板谈判未成，遂于当晚19时许至附近梅陇西路的“海友酒店”门口，无故围困住“志帆网吧”员工黄光兴。网吧领班被害人陈某某闻讯带领员工被害人谢某、余某某、解某某、任某某四人至“海友酒店”门前查看，随即遭到被告人何文军一伙无故殴打并不同程度受伤。经鉴定，被害人陈某某乳突部软组织挫伤、面部皮肤创、右上臂及右侧胸壁软组织挫伤，分别构成轻微伤；被害人谢某双眼部挫伤，构成轻微伤；被害人余某某左侧上颌骨额突骨折，构成轻微伤。\n嗣后，被告人何文军被公安机关抓获，其到案后在审查起诉阶段自愿如实供述了上述主要事实。案发后，被告人何文军取得五名被害人的谅解。\n为证实上述指控的事实，公诉机关当庭宣读、出示了被害人陈某某、谢某、余某某、解某某、任某某的陈述及有关辨认笔录，证人塔依尔·依明、黄光兴、王震的证言及有关辨认笔录，公安机关出具的验伤通知书、扣押决定书、扣押清单、“海友酒店”门前监控录像，上海市闵行区中心医院出具的医院检验情况记录，上海枫林司法鉴定有限公司出具的鉴定意见书，有关谅解书、相关刑事判决书，被告

detail_page:  {'s1': '摨全寻衅滋事一审刑事判决书', 's2': '上海市长宁区人民法院', 's3': '914', 's5': 'af310d51fe064cd1bdb0a97e00e61a20', 's6': '01', 's7': '（2018）沪0105刑初914号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-08-30', 's41': '2018-10-22', 's22': '上海市长宁区人民法院\n刑事判决书\n（2018）沪0105刑初914号', 's23': '上海市长宁区人民检察院以沪长检诉刑诉［2018］894号起诉书指控被告人周全犯寻衅滋事罪，于2018年8月21日向本院提起公诉。本院适用刑事案件速裁程序，实行独任审判，公开开庭审理了本案', 's25': '上海市长宁区人民检察院指控，2018年6月30日2时许，被告人周全在本市普陀区吉镇路、清峪路路口乘坐被害人潘某某驾驶的滴滴专车至本市长宁区天山路、威宁路公交车站附近时，因被告人周全认为潘某某绕道行驶，双方发生口角，随后，被告人周全拳击被害人潘某某面部致其受伤。经法医鉴定，被害人潘某某的伤势构成轻伤二级。\n2018年7月11日，被告人周全接民警电话通知后，主动至派出所投案，并如实供述自己的犯罪事实。被告人周全到案后已赔偿被害人损失并获得被害人的谅解。\n上海市长宁区人民检察院认为被告人周全具有自首的从轻处罚情节，建议判处被告人周全拘役之刑罚。\n被告人周全对指控事实、证据、罪名及量刑建议没有异议且签字具结，在开庭审理过程中亦无异议', 's26': '本院认为，公诉机关指控被告人周全犯寻衅滋事罪的事实清楚，证据确实、充分，指控罪名成立，量刑建议适当，应予采纳。辩护人以被告人周全系自首，且赔偿被害人并获得谅解，依法可以从轻处罚的意见，本院予以采纳。为维护社会治安管理秩序，依照《中华人民共和国刑法》第二百九十三条第一款、第六十七条第一款，第七十二条第一款，第七十三条第一款及第三款之规定，判决如下', 's27': '被告人周全犯寻衅滋事罪，判处拘役三个月，缓刑三个月。\n（缓刑考验期限从判决确定之日起计算。）\n周全回到社区后，应当遵守法律、法规，服从监督管理，接受教育，完成公益劳动，做一名有益社会的公民。\n如不服本判决

detail_page:  {'s1': '䟳亚军寻衅滋事罪一案二审刑事裁定书', 's2': '上海市第一中级人民法院', 's3': '910', 's5': '88a30425b3984f128bf5a9a100ae4547', 's6': '02', 's7': '（2018）沪01刑终1261号', 's8': '刑事案件', 's9': '刑事二审', 's31': '2018-10-08', 's41': '2019-01-14', 's22': '上海市第一中级人民法院\n刑事裁定书\n（2018）沪01刑终1261号', 's23': '上海市徐汇区人民法院审理上海市徐汇区人民检察院指控原审被告人石亚军犯寻衅滋事罪一案，于二〇一八年五月二十九日作出（2018）沪0104刑初260号刑事判决，对被告人石亚军犯寻衅滋事罪，判处有期徒刑一年。原审被告人石亚军不服，提出上诉。本院依法组成合议庭进行了审理。本院审理过程中，上诉人石亚军申请撤回上诉', 's26': '本院认为，原审判决认定上诉人石亚军犯寻衅滋事罪的事实清楚，证据确实、充分，适用法律正确，量刑适当，审判程序合法。依照最高人民法院《关于适用〈中华人民共和国刑事诉讼法〉的解释》第三百零五条第一款之规定，裁定如下', 's27': '准许上诉人石亚军撤回上诉。\n上海市徐汇区人民法院（2018）沪0104刑初260号刑事判决自本裁定送达之日起发生法律效力。\n本裁定为终审裁定', 's28': '审判长沈黎\n审判员郑焯琼\n审判员曹延\n二〇一八年十月八日\n书记员万兆晴', 's51': '附：相关法律条文\n《最高人民法院关于适用＜中华人民共和国刑事诉讼法＞的解释》第三百零五条上诉人在上诉期满后要求撤回上诉的，第二审人民法院应当审查。经审查，认为原判认定事实和适用法律正确，量刑适当的，应当裁定准许撤回上诉；认为原判事实不清、证据不足或者将无罪判为有罪、轻罪重判等的，应当不予准许，继续按照上诉案件审理', 's17': ['石亚军'], 's45': ['寻衅滋事', '程序合法'], 's11': ['寻衅滋事'], 'wenshuAy': [{'key': 's14', 'value': '233', 'text': '寻衅滋事'}], 's47': [{'tkx': '第三百零五条第一款'

detail_page:  {'s1': '䎋某某、赵某某寻衅滋事一审刑事判决书', 's2': '上海市浦东新区人民法院', 's3': '916', 's5': '5f92df4d9be5461fa66eaa7800f721ab', 's6': '01', 's7': '（2018）沪0115刑初4689号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-12-25', 's41': '2019-06-28', 's22': '上海市浦东新区人民法院\n刑事判决书\n（2018）沪0115刑初4689号', 's23': '上海市浦东新区人民检察院以沪浦检张江刑诉［2018］4281号起诉书指控被告人王某某、赵某某犯寻衅滋事罪，于2018年11月28日向本院提起公诉。本院于同年12月1日立案后，依法组成合议庭，公开开庭审理了本案。上海市浦东新区人民检察院指派检察员陈某某、井某某出庭支持公诉，上列被告人及辩护人均到庭参加诉讼。现已审理终结', 's25': '上海市浦东新区人民检察院指控被告人王某某、赵某某伙同他人于2018年8月19日21时50分许在本区川沙新镇象山海鲜酒店门口，因代驾生意与周某发生冲突，殴打周某致其轻伤；为证实上述指控，公诉机关提供了相应证据，据此指控被告人王某某、赵某某的行为均已触犯《中华人民共和国刑法》第二百九十三条第一款第一项、第二十五条第一款的规定，应当以寻衅滋事罪追究刑事责任；认定被告人赵某某具有坦白情节，依照《中华人民共和国刑法》第六十七条第三款的规定可以从轻处罚。\n被告人王某某、赵某某对公诉机关指控的事实及罪名均无异议。\n辩护人张祝梅对公诉机关指控的事实及罪名均无异议，辩称被告人王某某系从犯，主观恶性较小，被害人有明显过错，愿意赔偿被害人损失，能自愿认罪，建议对其从轻处罚。\n辩护人沈建军对公诉机关指控的事实及罪名均无异议，辩称被告人赵某某能如实供述，被害人有一定过错，愿意赔偿，系初犯偶犯，建议法庭考虑其家庭情况，对其从轻处罚并适用缓刑。\n经审理查明：\n2018年8月19日21时50分许，被告人王某某、赵某某等人在本区川沙新镇华新路、新川路路口南侧的象山海鲜酒店门口因代驾顺序问题与周某发生冲突，后结伙将周某殴打致伤。\n经鉴定，周某双侧鼻骨骨折，已构成轻伤二级；面部挫伤，已构成轻微伤。

detail_page:  {'s1': 'Ꙇ永强寻衅滋事二审刑事裁定书', 's2': '上海市第二中级人民法院', 's3': '920', 's5': '72aa82a5a1034576acc6a89600a5b9d5', 's6': '02', 's7': '（2018）沪02刑终132号', 's8': '刑事案件', 's9': '刑事二审', 's31': '2018-02-02', 's41': '2018-03-02', 's22': '上海市第二中级人民法院\n刑事裁定书\n（2018）沪02刑终132号', 's23': '上海市青浦区人民法院审理上海市青浦区人民检察院指控原审被告人陆永强犯寻衅滋事罪一案，于2017年11月21日作出（2017）沪0118刑初1150号刑事判决，对被告人陆永强犯寻衅滋事罪，判处有期徒刑十个月。原审被告人陆永强不服，提出上诉。本院依法组成合议庭，公开开庭审理了本案。上海市人民检察院第二分院指派检察员许某某出庭履行职务。上诉人陆永强到庭参加诉讼。在庭审中，上诉人陆永强申请撤回上诉', 's25': '上海市人民检察院第二分院认为，原判决认定原审被告人陆永强犯寻衅滋事罪的事实清楚，证据确实、充分，定罪量刑均无不当，诉讼程序合法。现陆永强自愿撤诉，并无不当，建议予以准许', 's26': '本院认为，原审法院认定原审被告人陆永强犯寻衅滋事罪的事实清楚，适用法律正确，量刑适当。上诉人陆永强申请撤回上诉，应予准许。上海市人民检察院第二分院意见正确。据此，依照《最高人民法院关于适用〈中华人民共和国刑事诉讼法〉的解释》第三百零五条第一款、第三百零八条的规定，裁定如下', 's27': '准许上诉人陆永强撤回上诉。\n上海市青浦区人民法院（2017）沪0118刑初1150号刑事判决自本裁定送达之日起发生法律效力。\n本裁定为终审裁定', 's28': '审判长董玮\n审判员袁婷\n代理审判员周广明\n二〇一八年二月二日\n书记员李姝', 's51': '附：相关法律条文\n附：相关的法律条文\n《最高人民法院关于适用的解释》\n第三百零五条上诉人在上诉期满后要求撤回上诉的，第二审人民法院应当审查。经审查，认为原判认定事实和适用法律正确，量刑适当的，应当裁定准许撤回上诉；认为原判事实不清、证据不足或者将无罪判为有罪、轻罪重判等的，

...

detail_page:  {'s1': '绘某某放火一审刑事判决书', 's2': '上海市浦东新区人民法院', 's3': '916', 's5': 'bb7abbafe2394ee8bcc3a9a100b30854', 's6': '01', 's7': '（2018）沪0115刑初1979号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-08-09', 's41': '2019-03-12', 's22': '上海市浦东新区人民法院\n刑事判决书\n（2018）沪0115刑初1979号', 's23': '上海市浦东新区人民检察院以沪浦检诉一刑诉〔2018〕2035号起诉书指控被告人付某某犯放火罪，于2018年5月18日向本院提起公诉。本院受理后，因发现有不事宜适用简易程序的情形，遂依法转为普通程序，组成合议庭，公开开庭审理了本案。上海市浦东新区人民检察院指派检察员丁某出庭支持公诉，被告人付某某及其辩护人盛立贞到庭参加诉讼。现已审理终结', 's25': '上海市浦东新区人民检察院指控：2018年1月5日14时42分许，被告人付某某在上海市浦东新区和炯路77弄盛世南苑小区送快递，期间其为发泄情绪，持打火机两次点燃11号楼2号正在运行中的电梯内壁上张贴的纸张，导致该电梯Ｐ1板毁损，物损价值人民币3700元，危及该楼196户人员安全。\n被告人付某某到案后，如实供述罪行。\n为证实上述指控，公诉机关当庭出示和宣读了被告人供述，证人证言，扣押决定书、扣押笔录及扣押清单，上海市公安局浦东分局接受证据清单，相关视频资料，上海市浦东新区价格认证中心价格认证结论书，户籍资料，案发经过等证据。公诉机关认为，被告人付某某以放火的方法危害公共安全，尚未造成严重后果，其行为已构成放火罪。被告人系坦白，可以从轻处罚。提请法院依法审判。\n被告人付某某对公诉机关指控的事实无异议，但是认为其行为构成寻衅滋事罪。\n辩护人对公诉机关指控的事实无异议，但是认为被告人的行为构成寻衅滋事罪。同时希望法院对被告人适用缓刑。\n经审理查明，2018年1月5日14时42分许，被告人付某某在上海市浦东新区和炯路77弄盛世南苑小区送快递，期间其为发泄情绪，持打火机两次点燃11号楼2号正在运行中的电梯内壁上张贴的纸张，导致该电梯Ｐ1板毁损，物损价值人民币3700元，

detail_page:  {'s1': '꛷某某、李某等寻衅滋事一审刑事判决书', 's2': '上海市浦东新区人民法院', 's3': '916', 's5': '1f455f5a82844320a025a8b50015fa04', 's6': '01', 's7': '（2018）沪0115刑初144号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-02-05', 's41': '2018-04-01', 's22': '上海市浦东新区人民法院\n刑事判决书\n（2018）沪0115刑初144号', 's23': '上海市浦东新区人民检察院以沪浦检刑诉〔2018〕98号起诉书指控被告人雷某某、李某、陶某、彭某某犯寻衅滋事罪，于2018年1月3日向本院提起公诉。本院于次日立案，依法适用简易程序，组成合议庭，公开开庭审理了本案。上海市浦东新区人民检察院指派检察员成某某出庭支持公诉，被告人雷某某、李某、陶某、彭某某及辩护人张广朋、姬金涛、徐国春、刘德平到庭参加诉讼。现已审理终结', 's25': '经审理查明，2017年9月23日凌晨2时许，被告人雷某某驾车载被告人李某等人经停本区浙桥路ＸＸＸ号门前时，因在该处吃夜宵的金某无故踢其驾驶车辆左前部而发生口角，在金某同行人员何某、秦某某等人调停下，被告人雷某某驾车离开。后被告人雷某某纠集被告人陶某，被告人李某纠集被告人彭某某，四人先后至浙桥路ＸＸＸ号门前，持就地取得的木凳等物殴打秦某某、金某、何某、钱某某致伤。经鉴定，被害人秦某某的伤势分别已构成轻伤二级、轻微伤；被害人金某的伤势已构成轻微伤；被害人何某的伤势已构成轻微伤；被害人钱某某的伤势尚未构成轻微伤。\n2017年9月23日11时许，被告人雷某某、李某、陶某、彭某某被公安机关抓获归案，到案后均如实供述上述犯罪事实。\n本院审理期间，四名被告人在家属帮助下与被害人秦某某、金某、何某达成了民事赔偿协议，支付了全部赔偿款人民币25万元，取得谅解，且三名被害人表示同意对被告人李某、彭某某宣告缓刑。\n上述事实，被告人雷某某、李某、陶某、彭某某在开庭审理过程中亦无异议，且有经庭审质证属实的被害人秦某某、金某、何某、钱某某的陈述及相关辨认笔录，证人陈某某、程某某的证言笔录，验伤通知书、司法鉴定意见书，公安机关的现场勘验笔录、ＤＮＡ检验鉴定

detail_page:  {'s1': '殣2、杨某等以危险方法危害公共安全一审刑事判决书', 's2': '上海市浦东新区人民法院', 's3': '916', 's5': '19cf368dc04940ada63da9e600f93efc', 's6': '01', 's7': '（2018）沪0115刑初2826号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-10-17', 's41': '2019-02-18', 's22': '上海市浦东新区人民法院\n刑事判决书\n（2018）沪0115刑初2826号', 's23': '上海市浦东新区人民检察院以沪浦检诉一刑诉〔2018〕2955号起诉书指控被告人宣2、杨某犯以危险方法危害公共安全罪、被告人张某某犯以危险方法危害公共安全罪、寻衅滋事罪，于2018年7月19日向本院提起公诉。本院于同日立案后，依法适用普通程序，组成合议庭，公开开庭审理了本案。上海市浦东新区人民检察院指派检察员王某某出庭支持公诉，上列被告人及辩护人到庭参加诉讼。现已审理终结', 's25': '上海市浦东新区人民检察院指控：\n2016年12月18日凌晨，被告人张某某在上海市闵行区中华美路北翟路口附近因琐事与被害人韩某发生纠纷，动手殴打韩某，后在被害人韩某前来讨要说法时，伙同他人对韩某拳打脚踢致伤。经鉴定，被害人韩某的伤势构成轻伤。\n2017年11月7日凌晨，被告人杨某和刁某某（另行处理）在上海市武宁路、长寿路口声称其牌号为苏Ａ7ＸＸＸＸ的起亚轿车遭到被告人宣2驾驶的车辆碰擦。宣2担心自己驾驶的套牌出租车遇上“碰瓷”敲诈，便驾车逃离。刁某某遂先行驾车追逐，再由杨某纠集被告人张某某驾驶套牌出租车共同赶至浦东新区。在本区张杨北路、洲海路附近，张某某、刁某某驾车将宣2驾驶的车辆逼停，刁某某阻挡在宣2的车辆前方，宣2见状将刁某某顶在引擎盖及车顶，再次驾车沿张杨北路逆向行驶超速逃离，杨某、张某某亦驾车逆向行驶超速追逐。在翔殷路隧道附近，杨某使用工具将宣2已停驶车辆的玻璃砸碎，后宣2再次驾车逃离，杨某仅仅跟随，张某某亦尾随而来。行至本区高行镇东塘路ＸＸＸ号附近时，宣2驾车撞上路边设施，致刁某某从车上甩出摔伤。杨某驾驶的车辆撞上宣2的车辆，造成二车不同程度损坏。期间，被告人宣2、杨某、张某某驾车追逐全长32.

detail_page:  {'s1': '껄某某寻衅滋事一审刑事判决书', 's2': '上海市静安区人民法院', 's3': '922', 's5': 'a1ed4a31b4be427782b8a91901270474', 's6': '01', 's7': '（2018）沪0106刑初288号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-04-04', 's41': '2019-03-04', 's22': '上海市静安区人民法院\n刑事判决书\n（2018）沪0106刑初288号', 's23': '上海市静安区人民检察院以沪静检诉刑诉［2018］278号起诉书指控被告人黄某某犯寻衅滋事罪，于2018年3月12日向本院提起公诉。本院受理后，依法组成合议庭，公开开庭审理了本案。上海市静安区人民检察院指派检察员徐衍出庭支持公诉。被告人黄某某及辩护人徐以琴到庭参加诉讼。现已审理终结', 's25': '公诉机关指控，被告人黄某某于2016年11月至2017年9月，多次至本市静安区中华新路ＸＸＸ号快客超市内，无故辱骂、恐吓被害人沈某某。期间，黄某某于2016年12月30日已因上述行为引发的妨害公务罪被判处刑罚，但在刑满释放后，黄某某继续对沈某某进行辱骂、恐吓。此外，黄某某还多次拨打12345市民热线电话扬言要杀害沈某某及相关民警。2017年9月24日，被告人黄某某被民警抓获。\n对于上述指控的事实，公诉机关提供了相关证人证言、书证等证据，认为被告人黄某某多次辱骂、恐吓他人，其行为已构成寻衅滋事罪。提请依照《中华人民共和国刑法》第二百九十三条之规定，依法追究被告人黄某某的刑事责任。\n被告人黄某某否认起诉指控的犯罪事实和罪名，辩称自己与被害人之前认识，后有过吵闹，被害人也找人打过其耳光，自己并非无故，也没有辱骂过被害人。其辩护人对起诉指控的犯罪事实和罪名均无异议，但提出被告人黄某某与被害人之前就认识，其辱骂、恐吓行为主要集中在2016年12月30日前，被判处刑罚后再没有寻衅滋事的行为，公诉机关指控的犯罪区间过长，请法院综合考虑本案的事实和后果，对被告人黄某某从轻处罚。\n经审理查明，被告人黄某某于2016年11月至2017年9月，多次至本市静安区中华新路ＸＸＸ号快客超市内，无故辱骂、恐吓被害人沈某某。期间，黄某某于2016年1

detail_page:  {'s1': 'ꭏ某某、刘某寻衅滋事一审刑事判决书', 's2': '上海市浦东新区人民法院', 's3': '916', 's5': 'f54dfe3a5bfc4b0995bdaa0e00ffdd8d', 's6': '01', 's7': '（2018）沪0115刑初4679号', 's8': '刑事案件', 's9': '刑事一审', 's31': '2018-12-14', 's41': '2019-05-05', 's22': '上海市浦东新区人民法院\n刑事判决书\n（2018）沪0115刑初4679号', 's23': '上海市浦东新区人民检察院以沪浦检张江刑诉［2018］4242号起诉书指控被告人魏某某、刘某犯寻衅滋事罪，于2018年11月26日向本院提起公诉。本院于同年12月1日立案后，依法适用简易程序，组成合议庭，公开开庭审理了本案。上海市浦东新区人民检察院指派检察员吴某某出庭支持公诉，二名被告人及其辩护人均到庭参加诉讼。现已审理终结', 's25': '经审理查明：2018年7月25日4时许，被告人魏某某、刘某等人在上海市浦东新区川沙新镇川黄路ＸＸＸ号格老子火锅店，由被告人魏某某起意并用皮带抽打陈某，被告人刘某等人亦参与殴打陈某。经鉴定，被害人陈某的伤势构成轻微伤。\n2018年8月3日，被告人魏某某被公安机关抓获；同月21日，被告人刘某自动向公安机关投案，二名被告人到案后均如实供述了上述罪行。\n上述事实，二名被告人在开庭审理过程中亦无异议，并有公诉机关提供且均经庭审质证属实的二名被告人的供述笔录，被害人陈某的陈述笔录，证人宋某某、王某1、苏某、王某2、张某某的证言笔录，有关监控录像、验伤通知书、伤势鉴定意见书，公安机关的案发经过记录，被告人刘某的前科材料等证据予以证实，足以认定', 's26': '本院认为，被告人魏某某、刘某在公共场所随意殴打他人，情节恶劣，破坏社会秩序，依照《中华人民共和国刑法》（以下简称《刑法》）第二百九十三条第一款第（一）项、第二十五条第一款的规定，均已构成寻衅滋事罪，分别应处五年以下有期徒刑。公诉机关指控二名被告人犯寻衅滋事罪的事实清楚，证据确实、充分，罪名成立。被告人刘某曾因故意犯罪被判处有期徒刑，刑罚执行完毕后五年内又重新故意犯应当判处有期徒刑之罪，依照《刑法》第六十五条第一款的规定